*If you see ![](http://training.databricks.com/databricks_guide/ImportNotebookIcon3.png) at the top-left or top-right, click on the link to import this notebook in order to run it.*

# On-Time Flight Performance with GraphFrames for Apache Spark
This notebook provides an analysis of On-Time Flight Performance and Departure Delays data using GraphFrames for Apache Spark.  This notebook has been updated to use Apache Spark 2.0 and GraphFrames 0.3.
* Original blog post: [On-Time Flight Performance with GraphFrames with Apache Spark Blog Post](https://databricks.com/blog/2016/03/16/on-time-flight-performance-with-graphframes-for-apache-spark.html)
* Original Notebook: [On-Time Flight Performance with GraphFrames with Apache Spark Notebook](http://cdn2.hubspot.net/hubfs/438089/notebooks/Samples/Miscellaneous/On-Time_Flight_Performance.html)


Source Data: 
* [OpenFlights: Airport, airline and route data](http://openflights.org/data.html)
* [United States Department of Transportation: Bureau of Transportation Statistics (TranStats)](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time)
 * Note, the data used here was extracted from the US DOT:BTS between 1/1/2014 and 3/31/2014*

References:
* [GraphFrames User Guide](http://graphframes.github.io/user-guide.html)
* [GraphFrames: DataFrame-based Graphs (GitHub)](https://github.com/graphframes/graphframes)
* [D3 Airports Example](http://mbostock.github.io/d3/talk/20111116/airports.html)
* [MLlib and Machine Learning: Binary Classification](https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html)

### Preparation
Extract the Airports and Departure Delays information from S3 / DBFS

In [1]:
import os
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import udf

In [2]:
sc = pyspark.SparkContext("local[*]")
spark = SparkSession.builder.appName('airlines_assignment_template').getOrCreate()

In [3]:
from graphframes import *
import hashlib

In [4]:
# Set File Paths
tripdelaysFilePath = "departuredelays.csv"
airportsnaFilePath = "airport-codes-na.txt"

# Obtain airports dataset
airportsna = spark.read.csv(airportsnaFilePath, header='true', inferSchema='true', sep='\t')
airportsna.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
departureDelays = spark.read.csv(tripdelaysFilePath, header='true')
departureDelays.createOrReplaceTempView("departureDelays")
departureDelays.cache()

# Available IATA codes from the departuredelays sample dataset
tripIATA = spark.sql("select distinct iata from (select distinct origin as iata from departureDelays union all select distinct destination as iata from departureDelays) a")
tripIATA.createOrReplaceTempView("tripIATA")

# Only include airports with atleast one trip from the departureDelays dataset
airports = spark.sql("select f.IATA, f.City, f.State, f.Country from airports_na f join tripIATA t on t.IATA = f.IATA")
airports.createOrReplaceTempView("airports")
airports.cache()

DataFrame[IATA: string, City: string, State: string, Country: string]

In [5]:
departureDelays.count()

1391578

In [6]:
# Build `departureDelays_geo` DataFrame
#  Obtain key attributes such as Date of flight, delays, distance, and airport information (Origin, Destination)  
departureDelays_geo = spark.sql("select cast(f.date as int) as tripid, cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ' '), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, cast(f.delay as int), cast(f.distance as int), f.origin as src, f.destination as dst, o.city as city_src, d.city as city_dst, o.state as state_src, d.state as state_dst from departuredelays f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# Create Temporary View and cache
departureDelays_geo.createOrReplaceTempView("departureDelays_geo")
departureDelays_geo.cache()

# Count
departureDelays_geo.count()

1361141

In [7]:
# Review the top 10 rows of the `departureDelays_geo` DataFrame
departureDelays_geo.show(10)

+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
| tripid|          localdate|delay|distance|src|dst|   city_src|           city_dst|state_src|state_dst|
+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
|1011111|2014-01-01 11:11:00|   -5|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1021111|2014-01-02 11:11:00|    7|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1031111|2014-01-03 11:11:00|    0|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1041925|2014-01-04 19:25:00|    0|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1061115|2014-01-06 11:15:00|   33|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1071115|2014-01-07 11:15:00|   23|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1081115|2014-01-08 11:15:00|   -9|     221|MSP|INL|Min

In [8]:
# Using `display` to view the data
# print(departureDelays_geo)
departureDelays_geo.show(3)

+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
| tripid|          localdate|delay|distance|src|dst|   city_src|           city_dst|state_src|state_dst|
+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
|1011111|2014-01-01 11:11:00|   -5|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1021111|2014-01-02 11:11:00|    7|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|1031111|2014-01-03 11:11:00|    0|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
only showing top 3 rows



## Building the Graph
Now that we've imported our data, we're going to need to build our graph. To do so we're going to do two things: we are going to build the structure of the vertices (or nodes) and we're going to build the structure of the edges. What's awesome about GraphFrames is that this process is incredibly simple. 
* Rename IATA airport code to **id** in the Vertices Table
* Start and End airports to **src** and **dst** for the Edges Table (flights)

These are required naming conventions for vertices and edges in GraphFrames as of the time of this writing (Feb. 2016).

**WARNING:** If the graphframes package, required in the cell below, is not installed, follow the instructions [here](http://cdn2.hubspot.net/hubfs/438089/notebooks/help/Setup_graphframes_package.html).

In [9]:
# Note, ensure you have already installed the GraphFrames spack-package
from pyspark.sql.functions import *
from graphframes import *

# Create Vertices (airports) and Edges (flights)
tripVertices = airports.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays_geo.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache Vertices and Edges
tripEdges.cache()
tripVertices.cache()

DataFrame[id: string, City: string, State: string, Country: string]

In [10]:
# Vertices
#   The vertices of our graph are the airports
#display(tripVertices)
tripVertices.show(5)

+---+-------------------+-----+-------+
| id|               City|State|Country|
+---+-------------------+-----+-------+
|INL|International Falls|   MN|    USA|
|MSY|        New Orleans|   LA|    USA|
|GEG|            Spokane|   WA|    USA|
|BUR|            Burbank|   CA|    USA|
|SNA|      Orange County|   CA|    USA|
+---+-------------------+-----+-------+
only showing top 5 rows



In [11]:
# Edges
#  The edges of our graph are the flights between airports
#display(tripEdges)
tripEdges.show(5)

+-------+-----+---+---+-------------------+---------+
| tripid|delay|src|dst|           city_dst|state_dst|
+-------+-----+---+---+-------------------+---------+
|1011111|   -5|MSP|INL|International Falls|       MN|
|1021111|    7|MSP|INL|International Falls|       MN|
|1031111|    0|MSP|INL|International Falls|       MN|
|1041925|    0|MSP|INL|International Falls|       MN|
|1061115|   33|MSP|INL|International Falls|       MN|
+-------+-----+---+---+-------------------+---------+
only showing top 5 rows



In [12]:
spark

In [13]:
# Build `tripGraph` GraphFrame
#  This GraphFrame builds up on the vertices and edges based on our trips (flights)
tripGraph = GraphFrame(tripVertices, tripEdges)

# Build `tripGraphPrime` GraphFrame
#   This graphframe contains a smaller subset of data to make it easier to display motifs and subgraphs (below)
tripEdgesPrime = departureDelays_geo.select("tripid", "delay", "src", "dst")
tripGraphPrime = GraphFrame(tripVertices, tripEdgesPrime)

Py4JJavaError: An error occurred while calling o73.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(

## Simple Queries
Let's start with a set of simple graph queries to understand flight performance and departure delays

#### Determine the number of airports and trips

In [ ]:
print("Airports: %d" % tripGraph.vertices.count())
print("Trips: %d" % tripGraph.edges.count())


#### Determining the longest delay in this dataset

In [ ]:
tripGraph.edges.groupBy().max("delay").show()

In [ ]:
# Finding the longest Delay
longestDelay = tripGraph.edges.groupBy().max("delay")
#display(longestDelay)
longestDelay.show()

#### Determining the number of delayed vs. on-time / early flights

In [ ]:
# Determining number of on-time / early flights vs. delayed flights
print("On-time / Early Flights: %d" % tripGraph.edges.filter("delay <= 0").count())
print("Delayed Flights: %d" % tripGraph.edges.filter("delay > 0").count())

#### What flights departing SEA are most likely to have significant delays. Choose your Airport instead of SEA (Seattle)
Note, delay can be <= 0 meaning the flight left on time or early

In [ ]:
tripGraph.edges\
  .filter("src = 'SEA' and delay > 0")\
  .groupBy("src", "dst")\
  .avg("delay")\
  .sort(desc("avg(delay)"))\
  .show(5)
# display(tripGraph.edges.filter("src = 'SEA' and delay > 0").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)")))

#### What destinations tend to have delays

In [ ]:
# After displaying tripDelays, use Plot Options to set `state_dst` as a Key.
tripDelays = tripGraph.edges.filter("delay > 0")
#display(tripDelays)
tripDelays.show(5)

#### What destinations tend to have significant delays departing from SEA

In [ ]:
# States with the longest cumulative delays (with individual delays > 100 minutes) (origin: Seattle)
#display(tripGraph.edges.filter("src = 'SEA' and delay > 100"))
tripGraph.edges.filter("src = 'SEA' and delay > 100").show(5)

## Vertex Degrees
* `inDegrees`: Incoming connections to the airport
* `outDegrees`: Outgoing connections from the airport 
* `degrees`: Total connections to and from the airport

Reviewing the various properties of the property graph to understand the incoming and outgoing connections between airports.

In [ ]:
# Degrees
#  The number of degrees - the number of incoming and outgoing connections - for various airports within this sample dataset
# display(tripGraph.degrees.sort(desc("degree")).limit(20))
tripGraph.degrees.sort(desc("degree")).show(20)

In [ ]:
# inDegrees
#  The number of degrees - the number of incoming connections - for various airports within this sample dataset
# display(tripGraph.inDegrees.sort(desc("inDegree")).limit(20))
tripGraph.inDegrees.sort(desc("inDegree")).show(20)

In [ ]:
# outDegrees
#  The number of degrees - the number of outgoing connections - for various airports within this sample dataset
display(tripGraph.outDegrees.sort(desc("outDegree")).limit(20))
tripGraph.outDegrees.sort(desc("outDegree")).show(20)

## City / Flight Relationships through Motif Finding
To more easily understand the complex relationship of city airports and their flights with each other, we can use motifs to find patterns of airports (i.e. vertices) connected by flights (i.e. edges). The result is a DataFrame in which the column names are given by the motif keys.

#### What delays might we blame on SFO

In [ ]:
# Using tripGraphPrime to more easily display 
#   - The associated edge (ab, bc) relationships 
#   - With the different the city / airports (a, b, c) where SFO is the connecting city (b)
#   - Ensuring that flight ab (i.e. the flight to SFO) occured before flight bc (i.e. flight leaving SFO)
#   - Note, TripID was generated based on time in the format of MMDDHHMM converted to int
#       - Therefore bc.tripid < ab.tripid + 10000 means the second flight (bc) occured within approx a day of the first flight (ab)
# Note: In reality, we would need to be more careful to link trips ab and bc.
motifs = tripGraphPrime.find("(a)-[ab]->(b); (b)-[bc]->(c)")\
  .filter("(b.id = 'SFO') and (ab.delay > 500 or bc.delay > 500) and bc.tripid > ab.tripid and bc.tripid < ab.tripid + 10000")
motifs.show()

## Determining Airport Ranking using PageRank
There are a large number of flights and connections through these various airports included in this Departure Delay Dataset.  Using the `pageRank` algorithm, Spark iteratively traverses the graph and determines a rough estimate of how important the airport is.

In [ ]:
# Determining Airport ranking of importance using `pageRank`
ranks = tripGraph.pageRank(resetProbability=0.15, maxIter=5)
#display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(10))
ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).show(10)

## Most popular flights (single city hops)
Using the `tripGraph`, we can quickly determine what are the most popular single city hop flights

In [ ]:
# Determine the most popular flights (single city hops)
import pyspark.sql.functions as func
topTrips = tripGraph \
  .edges \
  .groupBy("src", "dst") \
  .agg(func.count("delay").alias("trips")) 

In [ ]:
# Show the top 20 most popular flights (single city hops)
# display(topTrips.orderBy(topTrips.trips.desc()).limit(10))
topTrips.orderBy(topTrips.trips.desc()).show(10)

## Top Transfer Cities
Many airports are used as transfer points instead of the final Destination.  An easy way to calculate this is by calculating the ratio of inDegree (the number of flights to the airport) / outDegree (the number of flights leaving the airport).  Values close to 1 may indicate many transfers, whereas values < 1 indicate many outgoing flights and > 1 indicate many incoming flights.  Note, this is a simple calculation that does not take into account of timing or scheduling of flights, just the overall aggregate number within the dataset.

In [ ]:
# Calculate the inDeg (flights into the airport) and outDeg (flights leaving the airport)
inDeg = tripGraph.inDegrees
outDeg = tripGraph.outDegrees

# Calculate the degreeRatio (inDeg/outDeg)
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id) \
  .drop(outDeg.id) \
  .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio") \
  .cache()

# Join back to the `airports` DataFrame (instead of registering temp table as above)
nonTransferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio < .9 or degreeRatio > 1.1")

# List out the city airports which have abnormal degree ratios.
# display(nonTransferAirports)
nonTransferAirports.show()

In [ ]:
# Join back to the `airports` DataFrame (instead of registering temp table as above)
transferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio between 0.9 and 1.1")
  
# List out the top 10 transfer city airports
# display(transferAirports.orderBy("degreeRatio").limit(10))
transferAirports.orderBy("degreeRatio").show(10)

## Breadth First Search 
Breadth-first search (BFS) is designed to traverse the graph to quickly find the desired vertices (i.e. airports) and edges (i.e flights).  Let's try to find the shortest number of connections between cities based on the dataset.  Note, these examples do not take into account of time or distance, just hops between cities.

In [ ]:
# Example 1: Direct Seattle to San Francisco . Change San Francisco (SFO) to your airport of choice
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SEA'",
  toExpr = "id = 'SFO'",
  maxPathLength = 1)
# display(filteredPaths)
filteredPaths.show()

As you can see, there are a number of direct flights between Seattle and San Francisco.

In [ ]:
# Example 2: Direct San Francisco and Buffalo. Change Buffalo (BUF) to your airport of choice
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 2)
#display(filteredPaths)
filteredPaths.show(10)

In [ ]:
# Display most popular layover cities by descending count
# display(filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).limit(10))
filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).show(10)

In [ ]:
# Example 2: Direct San Francisco and Buffalo
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 1)
# display(filteredPaths)
filteredPaths.show()

But there are no direct flights between San Francisco and Buffalo.

In [ ]:
# Example 2a: Flying from San Francisco to Buffalo
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 2)
# display(filteredPaths)
filteredPaths.show(10)

But there are flights from San Francisco to Buffalo with Minneapolis as the transfer point.  But what are the most popular layovers between `SFO` and `BUF`?

In [ ]:
# Display most popular layover cities by descending count
# display(filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).limit(10))
filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).show(10)

In [ ]:
dataset.count()